# Project Title
### Data Engineering Capstone Project

#### Project Summary
--describe your project at a high level--

The project follows the follow steps:
* Step 1: Scope the Project and Gather Data
* Step 2: Explore and Assess the Data
* Step 3: Define the Data Model
* Step 4: Run ETL to Model the Data
* Step 5: Complete Project Write Up

In [1]:
# Do all imports and installs here
import pandas as pd
from pyspark.sql import SparkSession

### Step 1: Scope the Project and Gather Data

#### Scope 
Explain what you plan to do in the project in more detail. What data do you use? What is your end solution look like? What tools did you use? etc>

#### Describe and Gather Data 
Describe the data sets you're using. Where did it come from? What type of information is included? 

#### 1. Importing Immigration dataset:
*Immigration dataset if quite large in size so its better to use **Apache Spark** to read the files in this case.*

In [2]:
%%time
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()
df_immig =spark.read.format('com.github.saurfang.sas.spark').load('../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat')

CPU times: user 38.5 ms, sys: 23.2 ms, total: 61.6 ms
Wall time: 18 s


In [3]:
#write to parquet
#df_spark.write.parquet("sas_data")
#df_spark=spark.read.parquet("sas_data")

In [4]:
df_immig.take(2)

[Row(cicid=6.0, i94yr=2016.0, i94mon=4.0, i94cit=692.0, i94res=692.0, i94port='XXX', arrdate=20573.0, i94mode=None, i94addr=None, depdate=None, i94bir=37.0, i94visa=2.0, count=1.0, dtadfile=None, visapost=None, occup=None, entdepa='T', entdepd=None, entdepu='U', matflag=None, biryear=1979.0, dtaddto='10282016', gender=None, insnum=None, airline=None, admnum=1897628485.0, fltno=None, visatype='B2'),
 Row(cicid=7.0, i94yr=2016.0, i94mon=4.0, i94cit=254.0, i94res=276.0, i94port='ATL', arrdate=20551.0, i94mode=1.0, i94addr='AL', depdate=None, i94bir=25.0, i94visa=3.0, count=1.0, dtadfile='20130811', visapost='SEO', occup=None, entdepa='G', entdepd=None, entdepu='Y', matflag=None, biryear=1991.0, dtaddto='D/S', gender='M', insnum=None, airline=None, admnum=3736796330.0, fltno='00296', visatype='F1')]

#### 2. Importing world_temperature dataset:
*World temperature dataset if quite large in size so its better to use **Apache Spark** to read the files in this case.*

In [5]:
# %%time
# Read in the data here
# df_temp=pd.read_csv('../../data2/GlobalLandTemperaturesByCity.csv')

In [6]:
%%time
df_temp=spark.read.format('csv').option('header','True').load('../../data2/GlobalLandTemperaturesByCity.csv')

CPU times: user 2.79 ms, sys: 103 µs, total: 2.89 ms
Wall time: 2.8 s


In [7]:
df_temp.show(2)

+----------+------------------+-----------------------------+-----+-------+--------+---------+
|        dt|AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|             6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|              null|                         null|Århus|Denmark|  57.05N|   10.33E|
+----------+------------------+-----------------------------+-----+-------+--------+---------+
only showing top 2 rows



#### 3. Importing airport_code dataset:
*airport_code dataset is not that much large in size its better to use **Pandas** to read the files in this case.*

In [8]:
# Read in the data here
df_port=pd.read_csv('airport-codes_csv.csv')

In [9]:
df_port.head(2)

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
0,00A,heliport,Total Rf Heliport,11.0,NaN,US,US-PA,Bensalem,00A,NaN,00A,"-74.93360137939453, 40.07080078125"
1,00AA,small_airport,Aero B Ranch Airport,3435.0,NaN,US,US-KS,Leoti,00AA,NaN,00AA,"-101.473911, 38.704022"


In [10]:
df_port.query("iata_code=='MAA'")

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
51294,VOMM,large_airport,Chennai International Airport,52.0,AS,IN,IN-TN,Chennai,VOMM,MAA,NaN,"80.16929626464844, 12.990005493164062"


#### 4. Importing us_cities_demographics dataset:
*us_cities_demographics dataset is not that much large in size its better to use **Pandas** to read the files in this case.*

In [11]:
# Read in the data here
df_demog=pd.read_csv('us-cities-demographics.csv',delimiter=';')


In [12]:
df_demog.head(2)

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723


### Step 2: Explore and Assess the Data




### Explore the Data 
Identify data quality issues, like missing values, duplicate data, etc.

#### 1. Exploring Immigration dataset:

In [13]:
# Checking for missing values
print("Number of Columns: {}".format(len(df_immig.columns)))
print("Number of Rows: {}".format(df_immig.count()))

Number of Columns: 28
Number of Rows: 3096313


In [14]:
# Drop duplicates value
df_immig=df_immig.dropDuplicates()

In [15]:
# Number of rows after removing duplicates
print("Number of Rows: {}".format(df_immig.count()))

Number of Rows: 3096313


In [16]:
# Describe
df_immig.describe().show()

+-------+-----------------+-------+-------+------------------+------------------+-------+-----------------+------------------+------------------+-----------------+------------------+-------------------+-------+--------------------+--------+-----------------+-------+-------+-------+-------+------------------+------------------+-------+-----------------+------------------+--------------------+------------------+--------+
|summary|            cicid|  i94yr| i94mon|            i94cit|            i94res|i94port|          arrdate|           i94mode|           i94addr|          depdate|            i94bir|            i94visa|  count|            dtadfile|visapost|            occup|entdepa|entdepd|entdepu|matflag|           biryear|           dtaddto| gender|           insnum|           airline|              admnum|             fltno|visatype|
+-------+-----------------+-------+-------+------------------+------------------+-------+-----------------+------------------+------------------+---------

#### 2. Exploring world_temperature dataset:

In [17]:
#Shape of the Dataset
print("Number of Columns: {}".format(len(df_temp.columns)))
print("Number of Rows: {}".format(df_temp.count()))

Number of Columns: 7
Number of Rows: 8599212


In [18]:
#Shape of the dataset
df_temp.printSchema()

root
 |-- dt: string (nullable = true)
 |-- AverageTemperature: string (nullable = true)
 |-- AverageTemperatureUncertainty: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [19]:
# Checking missing values
df_temp.describe()

DataFrame[summary: string, dt: string, AverageTemperature: string, AverageTemperatureUncertainty: string, City: string, Country: string, Latitude: string, Longitude: string]

In [20]:
# Drop Duplicates Values
df_port.drop_duplicates(inplace=True)

#### 3. Exploring airport_code dataset:

In [21]:
#Shape of the dataset
df_port.shape

(55075, 12)

In [22]:
# Checking missing values
df_port.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55075 entries, 0 to 55074
Data columns (total 12 columns):
ident           55075 non-null object
type            55075 non-null object
name            55075 non-null object
elevation_ft    48069 non-null float64
continent       27356 non-null object
iso_country     54828 non-null object
iso_region      55075 non-null object
municipality    49399 non-null object
gps_code        41030 non-null object
iata_code       9189 non-null object
local_code      28686 non-null object
coordinates     55075 non-null object
dtypes: float64(1), object(11)
memory usage: 5.5+ MB


In [23]:
# Drop Duplicates Values
df_port.drop_duplicates(inplace=True)

#### 3. Exploring us_cities_demographics dataset:

In [24]:
# Shape of the dataset
df_demog.shape

(2891, 12)

In [25]:
# Checking for missing values
df_demog.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2891 entries, 0 to 2890
Data columns (total 12 columns):
City                      2891 non-null object
State                     2891 non-null object
Median Age                2891 non-null float64
Male Population           2888 non-null float64
Female Population         2888 non-null float64
Total Population          2891 non-null int64
Number of Veterans        2878 non-null float64
Foreign-born              2878 non-null float64
Average Household Size    2875 non-null float64
State Code                2891 non-null object
Race                      2891 non-null object
Count                     2891 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 271.1+ KB


In [26]:
# Dropping duplicates
df_demog.drop_duplicates(inplace=True)

#### Cleaning Steps
Document steps necessary to clean the data

In [27]:
# Performing cleaning tasks here





### Step 3: Define the Data Model
#### 3.1 Conceptual Data Model
Map out the conceptual data model and explain why you chose that model

#### 3.2 Mapping Out Data Pipelines
List the steps necessary to pipeline the data into the chosen data model

#### Using Star Schema:
##### Dimension Tables:
> 1. Immigrant Table
    * Columns: gender, biryear, occup, i94bir
##### Fact Table:

### Step 4: Run Pipelines to Model the Data 
#### 4.1 Create the data model
Build the data pipelines to create the data model.

In [30]:
# Write code here
immigrant_table=df_immig.select(['gender','biryear','occup','i94bir'])

In [41]:
immigrant_table.write.mode('overwrite').option('compression','snappy').parquet("immigrant")

In [42]:
df_spark=spark.read.option('compression','snappy').parquet("immigrant")

In [38]:
# Checking for missing values
print("Number of Columns: {}".format(len(df_spark.columns)))
print("Number of Rows: {}".format(df_spark.count()))

Number of Columns: 4
Number of Rows: 3096313


In [39]:
df_spark.show(5)

+------+-------+-----+------+
|gender|biryear|occup|i94bir|
+------+-------+-----+------+
|     M| 1954.0| null|  62.0|
|     F| 1979.0| null|  37.0|
|     F| 1997.0| null|  19.0|
|     F| 1947.0| null|  69.0|
|     M| 1971.0| null|  45.0|
+------+-------+-----+------+
only showing top 5 rows



#### 4.2 Data Quality Checks
Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:
 * Integrity constraints on the relational database (e.g., unique key, data type, etc.)
 * Unit tests for the scripts to ensure they are doing the right thing
 * Source/Count checks to ensure completeness
 
Run Quality Checks

In [29]:
# Perform quality checks here

### 4.3 Data dictionary 
_Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file._

#### Immigration Data:
~~~
  * cicid    -  
  * i94yr    - 4 digit year
  * i94mon   - Numeric month
  * i94cit   - This format shows all the valid and invalid codes for processing
  * i94res   - This format shows all the valid and invalid codes for processing
  * i94port  - This format shows all the valid and invalid codes for processing
  * arrdate  - is the Arrival Date in the USA. It is a SAS date numeric field that a 
               permament format has not been applied.  Please apply whichever date format 
               works for you.
  * i94mode  - There are missing values as well as not reported (9)
               	1 = 'Air'
                2 = 'Sea'
                3 = 'Land'
                9 = 'Not reported' ;   
  * i94addr  - There is lots of invalid codes in this variable and the list below 
               shows what we have found to be valid, everything else goes into 'other'
  * depdate  - is the Departure Date from the USA. It is a SAS date numeric field that 
               a permament format has not been applied.  Please apply whichever date format 
               works for you.
  * i94bir   - Age of Respondent in Years
  * i94visa  - Visa codes collapsed into three categories:
               1. 1 = Business
               2. 2 = Pleasure
               3. 3 = Student
  * count    - Used for summary statistics
  * dtadfile - Character Date Field - Date added to I-94 Files
  * visapost - Department of State where where Visa was issued
  * occup    - Occupation that will be performed in U.S.
  * entdepa  - Arrival Flag - admitted or paroled into the U.S.
  * entdepd  - Departure Flag - Departed, lost I-94 or is deceased
  * entdepu  - Update Flag - Either apprehended, overstayed, adjusted to perm residence
  * matflag  - Match flag - Match of arrival and departure records
  * biryear  - 4 digit year of birth
  * dtaddto  - Character Date Field - Date to which admitted to U.S. (allowed to stay until)
  * gender   - Non-immigrant sex
  * insnum   - INS number
  * airline  - Airline used to arrive in U.S.
  * admnum   - Admission Number
  * fltno    - Flight number of Airline used to arrive in U.S.
  * visatype - Class of admission legally admitting the non-immigrant to temporarily stay in U.S.
  
 ~~~

#### World Temperature Data:
```
* dt
* AverageTemperature
* AverageTemperatureUncertainty
* City
* Country
* Latitude
* Longitude
```

#### U.S. City Demographic Data:
```
* City
* State
* Median Age
* Male Population
* Female Population
* Total Population
* Number of Veterans
* Foreign-born
* Average Household Size
* State Code
* Race
* Count
```

#### Airport Codes:
```
* ident         - 
* type          - Type of airport e.g. 'small_airport', 'medium_airport', 'large_airport' etc
* name          - Airport full name.
* elevation_ft  - Elevation of the port measured in feet.  
* continent     - Continet code (e.g 'AS' for Asia,'NA' for North America)
* iso_country   - Two letter ISO country code assigned by International Organization for Standardization
* iso_region    - iso region code assigned by ISO.
* municipality  - Locality Municipality.  
* gps_code      - Global Positioning System code.
* iata_code     - IATA(Internation Air Transport Association) airport code, also known as IATA 
                  location identifier is a three letter code assigned to airports around the world.         
* local_code    - Similar to gps_code.
* coordinates   - (x,y) co-ordinates of the location.
```       

### Step 5: Complete Project Write Up
* Clearly state the rationale for the choice of tools and technologies for the project.
* Propose how often the data should be updated and why.
* Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 * The data populates a dashboard that must be updated on a daily basis by 7am every day.
 * The database needed to be accessed by 100+ people.